# Marginal Cost Calculation for Power Generators

This notebook demonstrates how to calculate the marginal cost for different power generators using real-world data. The workflow is organized into the following sections:


### 2. Preparing the Datasets
We clean and merge relevant columns from technology and emissions datasets into the main generator dataframe, and extract unique fuel and technology types.

### 3. Computing the Marginal Cost
We calculate fuel consumption, CO2 emissions, fuel cost, CO2 quota cost, operation & maintenance cost, total cost, and finally derive the marginal cost for each generator.

Follow the notebook step-by-step to understand how each component contributes to the marginal cost calculation.

### 1. Reading and Exploring Data
We load generator, technology, emissions, and fuel price data from CSV files and explore their contents.

### 1.1 Load generators file

In [1]:
import pandas as pd

df_raw = pd.read_csv("generators.csv", index_col=0)
df = df_raw.copy()
start_cols = df_raw.columns.tolist()
df.head()

,country,name,zone,fuel,fuel_type,technology,electric_capacity,thermal_capacity,efficiency,prod_cost
ID,,,,,,,,,,
G1BE,BE,Herdersbrug STEG,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,465.0,0.0,0.54,124.195170
G2BE,BE,Amercoeur 1 R TGV,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,451.0,0.0,0.54,127.012926
G3BE,BE,Aalter,BE,Gas oil,Gas oil - Decentral,Small CHP - Oil,18.0,0.0,0.42,401.649681
G4BE,BE,Tpower Beringen,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,425.0,0.0,0.54,125.513758
G5BE,BE,Marcinelle Energie Carsid,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,413.0,0.0,0.54,119.214277


In [2]:
fuel_list = list(df.fuel.unique())
fuel_list

['Natural gas',
 'Gas oil',
 'Waste',
 'Nuclear',
 'Wood pellets',
 'Wood chips',
 'Hydrogen',
 'Brown coal',
 'Coal',
 'Straw',
 'Biogas',
 'Fuel oil',
 'Peat',
 'Solar Thermal',
 'Tidal',
 'Geothermal']

In [3]:
technology_list = list(df.technology.unique())
technology_list

['Large CHP - natural gas CC',
 'Small CHP - Oil',
 'Medium CHP - waste',
 'Nuclear power plant',
 'Medium CHP - natural gas CC',
 'Large CHP - natural gas SC',
 'Medium CHP - natural gas SC',
 'Small CHP - waste',
 'Large CHP - wood pellets',
 'Medium CHP - wood pellets',
 'Engine - natural gas',
 'Large CHP - wood chips',
 'Large CHP - hydrogen CC',
 'Brown Coal Large CHP',
 'Large CHP - coal',
 'Medium CHP - wood chips',
 'Medium CHP - straw',
 'Large CHP - Oil',
 'Medium CHP - Oil',
 'Small CHP - wood chips',
 'Large CHP - waste',
 'Small CHP - straw',
 'Medium CHP - Peat',
 'Solar thermal electric',
 'Medium Tidal',
 'Geothermal electric',
 'Large CHP - Peat']

In [4]:
fuel_type_list = df.fuel_type.unique().tolist()
fuel_type_list

['Natural gas - Central',
 'Gas oil - Decentral',
 'Waste - Decentral',
 'Nuclear - Central',
 'Wood pellets - Central',
 'Natural gas - Decentral',
 'Wood chips - Central',
 'Hydrogen - Central',
 'Brown coal - Central',
 'Coal - Central',
 'Wood chips - Decentral',
 'Straw - Central',
 'Straw - Decentral',
 'Wood pellets - Decentral',
 'Gas oil - Central',
 'Biogas - Decentral',
 'Fuel oil - Central',
 'Peat - Central',
 'Solar Thermal - Central',
 'Tidal - Central',
 'Geothermal - Decentral',
 'Wood chips - Central - Finland']

### 1.2 Load technology file

In [5]:
df_tech_raw = pd.read_csv(
    "technology_data.csv", header=[0, 1], index_col=0, na_values=["---", "NA", "n/a"]
)
df_tech = df_tech_raw.copy()
df_tech.columns = df_tech.columns.get_level_values(0)
df_tech.head()

Technology,Reference,Capacity,Investment (el),Investment (heat),Fixed O&M (el),Fixed O&M (heat),Var. O&M (el),Var. O&M (heat),Lifetime,CV,...,Up-time,Down-time,Load factor,CO2 capture rate (amount of emission),SO2 (degree of desulphuring),Emission factor SO2,Emission factor NOX,Emission factor CH4,Emission factor N2O,Emission factor PM2.5
Small CHP - waste,Energistyrelsen,8.0,10.5,NaN,411000.0,NaN,25.9,NaN,25.0,1.0,...,8.0,8.0,NaN,0.0,1.00,NaN,241.2,0.36,3.6,1.08
Medium CHP - waste,Energistyrelsen,18.6,9.1,NaN,264800.0,NaN,25.0,NaN,25.0,1.0,...,8.0,8.0,NaN,0.0,1.00,NaN,201.6,0.36,3.6,1.08
Large CHP - waste,Energistyrelsen,51.8,7.8,NaN,188300.0,NaN,24.8,NaN,25.0,1.0,...,8.0,8.0,NaN,0.0,1.00,NaN,201.6,0.36,3.6,1.08
Boiler - waste,Energistyrelsen,NaN,NaN,1.75,NaN,78600.0,NaN,5.5,25.0,NaN,...,NaN,NaN,NaN,0.0,1.00,NaN,241.2,0.36,3.6,1.08
Small CHP - wood chips,Energistyrelsen,2.9,6.3,NaN,281000.0,NaN,9.3,NaN,25.0,1.0,...,8.0,8.0,NaN,0.0,0.98,NaN,226.8,39.60,3.6,1.08


### 1.3 Load emissions factor file

In [6]:
df_emissions_raw = pd.read_csv(
    "emissions.csv", header=[0, 1], index_col=0, na_values=["---", "NA", "n/a"]
)

df_emissions = df_emissions_raw.copy()
df_emissions.columns = df_emissions.columns.get_level_values(0)
df_emissions.head()

,co2_emission_pu,so2_emissions_pu
Natural gas,205.20,1.44
Biogas,0.00,0.00
Coal,338.40,28.80
Brown coal,364.32,2160.00
Fuel oil,284.40,360.00


### 2. Mapping the generators dataset
Now we have to link the technology catalogue to the plant in the datasets. We'll do that via a merge.

#### 2.1 Merging technology data

In [7]:
# Columns you want to bring in
cols_to_keep = [
    "Electric efficiency CHP",
    "Heat efficiency CHP",
    "Heat efficiency",
    "CO2 capture rate (amount of emission)",
    'Var. O&M (el)'
]

# Merge only these columns
df = df.merge(
    df_tech[cols_to_keep],  # select only needed columns
    left_on="technology",  # column in df
    right_index=True,  # df_tech index is plant_type
    how="left",
)
df.head()

,country,name,zone,fuel,fuel_type,technology,electric_capacity,thermal_capacity,efficiency,prod_cost,Electric efficiency CHP,Heat efficiency CHP,Heat efficiency,CO2 capture rate (amount of emission),Var. O&M (el)
ID,,,,,,,,,,,,,,,
G1BE,BE,Herdersbrug STEG,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,465.0,0.0,0.54,124.195170,0.54,0.30,NaN,0.0,4.4
G2BE,BE,Amercoeur 1 R TGV,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,451.0,0.0,0.54,127.012926,0.54,0.30,NaN,0.0,4.4
G3BE,BE,Aalter,BE,Gas oil,Gas oil - Decentral,Small CHP - Oil,18.0,0.0,0.42,401.649681,0.42,0.44,NaN,0.0,5.4
G4BE,BE,Tpower Beringen,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,425.0,0.0,0.54,125.513758,0.54,0.30,NaN,0.0,4.4
G5BE,BE,Marcinelle Energie Carsid,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,413.0,0.0,0.54,119.214277,0.54,0.30,NaN,0.0,4.4


#### 2.2 Merging emissions data

In [8]:
# Merge only these columns
df = df.merge(
    df_emissions[["co2_emission_pu", "so2_emissions_pu"]],  # select only needed columns
    left_on="fuel",  # column in df
    right_index=True,  # df_tech index is plant_type
    how="left",
)
df.head()

,country,name,zone,fuel,fuel_type,technology,electric_capacity,thermal_capacity,efficiency,prod_cost,Electric efficiency CHP,Heat efficiency CHP,Heat efficiency,CO2 capture rate (amount of emission),Var. O&M (el),co2_emission_pu,so2_emissions_pu
ID,,,,,,,,,,,,,,,,,
G1BE,BE,Herdersbrug STEG,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,465.0,0.0,0.54,124.195170,0.54,0.30,NaN,0.0,4.4,205.20,1.44
G2BE,BE,Amercoeur 1 R TGV,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,451.0,0.0,0.54,127.012926,0.54,0.30,NaN,0.0,4.4,205.20,1.44
G3BE,BE,Aalter,BE,Gas oil,Gas oil - Decentral,Small CHP - Oil,18.0,0.0,0.42,401.649681,0.42,0.44,NaN,0.0,5.4,266.76,82.80
G4BE,BE,Tpower Beringen,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,425.0,0.0,0.54,125.513758,0.54,0.30,NaN,0.0,4.4,205.20,1.44
G5BE,BE,Marcinelle Energie Carsid,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,413.0,0.0,0.54,119.214277,0.54,0.30,NaN,0.0,4.4,205.20,1.44


### 3. Calculating the marginal cost

The **marginal cost** of electricity generation for each power plant is the sum of three main components:

- **Fuel cost**: The cost of fuel consumed to produce electricity.
- **CO₂ quota cost**: The cost associated with carbon emissions, based on emission factors and quota price.
- **Variable Operation & Maintenance (O&M) cost**: The cost of operating and maintaining the plant per unit of electricity produced.

The equation for marginal cost ($MC$) is:

$$
MC = \frac{\text{Fuel Cost} + \text{CO}_2 \text{ Quota Cost} + \text{Variable O\&M Cost}}{\text{Electric Capacity}}
$$

Where:

- $\text{Fuel Cost} = \text{Fuel Consumption} \times \text{Fuel Price}$
- $\text{CO}_2 \text{ Quota Cost} = \text{CO}_2 \text{ Emission} \times \text{CO}_2 \text{ Quota Price}$
- $\text{Variable O\&M Cost} = \text{Var. O\&M (el)} \times \text{Electric Capacity}$

This calculation gives the marginal cost per unit of electricity generated for each plant.

#### 3.1 Calculation fuel consumption of full load

In [9]:
# Calculate fuel consumption (MWh-fuel per MWh-electric) using electric efficiency for CHP plants
df["fuel_consumption"] = df["electric_capacity"] / df["Electric efficiency CHP"]

# Calculate CO2 emissions, accounting for emission factor and CO2 capture rate
df["co2_emission"] = (
    df["fuel_consumption"]
    * df["co2_emission_pu"]
    * 1e-3  # convert from g/MWh-fuel to t/MWh-fuel
    * (1 - df["CO2 capture rate (amount of emission)"])  # adjust for capture rate
)

df.head()

,country,name,zone,fuel,fuel_type,technology,electric_capacity,thermal_capacity,efficiency,prod_cost,Electric efficiency CHP,Heat efficiency CHP,Heat efficiency,CO2 capture rate (amount of emission),Var. O&M (el),co2_emission_pu,so2_emissions_pu,fuel_consumption,co2_emission
ID,,,,,,,,,,,,,,,,,,,
G1BE,BE,Herdersbrug STEG,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,465.0,0.0,0.54,124.195170,0.54,0.30,NaN,0.0,4.4,205.20,1.44,861.111111,176.700000
G2BE,BE,Amercoeur 1 R TGV,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,451.0,0.0,0.54,127.012926,0.54,0.30,NaN,0.0,4.4,205.20,1.44,835.185185,171.380000
G3BE,BE,Aalter,BE,Gas oil,Gas oil - Decentral,Small CHP - Oil,18.0,0.0,0.42,401.649681,0.42,0.44,NaN,0.0,5.4,266.76,82.80,42.857143,11.432571
G4BE,BE,Tpower Beringen,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,425.0,0.0,0.54,125.513758,0.54,0.30,NaN,0.0,4.4,205.20,1.44,787.037037,161.500000
G5BE,BE,Marcinelle Energie Carsid,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,413.0,0.0,0.54,119.214277,0.54,0.30,NaN,0.0,4.4,205.20,1.44,764.814815,156.940000


#### 3.2 Extracting year of fuel projections

In [10]:
# Selection price year
price_year = 2040

df_fuel_price = pd.read_csv("fuel_price_projections.csv", index_col=0)
df_fuel_price_select = df_fuel_price[df_fuel_price.index == price_year].head()
df_fuel_price_select.head()

,CO2 quota,Coal - Central,Fuel oil - Central,Gas oil - Central,Gas oil - Decentral,Natural gas - Decentral,Natural gas - Central,Straw - Central,Straw - Decentral,Wood chips - CIF,...,Waste - Decentral,District heating - Decentral,Nuclear - Central,Peat - Central,Brown coal - Central,Geothermal - Decentral,Electricity - Decentral,Solar Thermal - Central,Excess Heat - Central,Tidal - Central
year,,,,,,,,,,,,,,,,,,,,,
2040,166.788571,10.0,118.0,118.0,118.0,25.0,25.0,29.0,27.0,33.0,...,0,0,4.0,18.0,4.0,0,0,0,0,0


#### 3.3 Calculating short run marginal costs

In [ ]:
# Map fuel prices and calculate marginal cost components
fuel_price_map = df_fuel_price_select.iloc[0].to_dict()  # Convert to dictionary for further use

# Map it into df
df['fuel_price'] = df['fuel_type'].map(fuel_price_map)


df['fuel_cost'] = df['fuel_consumption'] * df['fuel_price']
df['co2_quota_cost'] = df['co2_emission'] * fuel_price_map['CO2 quota']
df['om_cost'] = df['Var. O&M (el)'] * df['electric_capacity']
df['total_cost'] = df['fuel_cost'] + df['co2_quota_cost'] + df['om_cost']
df['marginal_cost'] = df['total_cost'] / df['electric_capacity']
df.head()

,country,name,zone,fuel,fuel_type,technology,electric_capacity,thermal_capacity,efficiency,prod_cost,...,co2_emission_pu,so2_emissions_pu,fuel_consumption,co2_emission,fuel_price,fuel_cost,co2_quota_cost,om_cost,total_cost,marginal_cost
ID,,,,,,,,,,,,,,,,,,,,,
G1BE,BE,Herdersbrug STEG,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,465.0,0.0,0.54,124.195170,...,205.20,1.44,861.111111,176.700000,25.0,21527.777778,29471.540549,2046.0,53045.318326,114.075953
G2BE,BE,Amercoeur 1 R TGV,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,451.0,0.0,0.54,127.012926,...,205.20,1.44,835.185185,171.380000,25.0,20879.629630,28584.225349,1984.4,51448.254979,114.075953
G3BE,BE,Aalter,BE,Gas oil,Gas oil - Decentral,Small CHP - Oil,18.0,0.0,0.42,401.649681,...,266.76,82.80,42.857143,11.432571,118.0,5057.142857,1906.822255,97.2,7061.165112,392.286951
G4BE,BE,Tpower Beringen,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,425.0,0.0,0.54,125.513758,...,205.20,1.44,787.037037,161.500000,25.0,19675.925926,26936.354265,1870.0,48482.280191,114.075953
G5BE,BE,Marcinelle Energie Carsid,BE,Natural gas,Natural gas - Central,Large CHP - natural gas CC,413.0,0.0,0.54,119.214277,...,205.20,1.44,764.814815,156.940000,25.0,19120.370370,26175.798380,1817.2,47113.368750,114.075953


#### 3.4 Export the file

In [12]:
df_out = df[start_cols + ['marginal_cost']].copy()
df_out.to_csv("generators_final.csv", index=True)